# Módulo 4: Compartir datos y pasos siguientes.


En esta sección examinaremos cómo alinear los genomas ensamblados, realizar análisis filogenéticos e interpretar árboles filogenéticos para investigaciones de brotes.


Como ha realizado antes:

1.) Haga clic en el archivo en la esquina superior izquierda y seleccione Guardar una copia en la unidad
Sus cambios no se guardarán si no hace este paso.

2.) Haga clic en el nombre del libro de trabajo en la esquina superior izquierda y reemplace "Copia de" con su nombre completo.

## **Parte 1**: Filogenética e interpretación

### Instalemos los paquetes de análisis necesarios.

Para esta parte usaremos: 
**seqtk** para evaluar la calidad del genoma (https://github.com/lh3/seqtk)  
**Mafft** para la alineación del genoma (https://mafft.cbrc.jp/alignment/software/)  
**snp-site** Para una evaluación rápida de de diferencia de SNP (https://github.com/sanger-pathogens/snp-sites)  
**Fasttree** para la construcción de árboles filogenéticos (http://www.microbesonline.org/fasttree/)  
**Phylo** d para visualización rápida de árboles con biophyton (https://biopython.org/wiki/Phylo). *Nota*: Hay muchos programas de Visualización de árboles. Los más utilizados comúnmente son ggtree (R), ete3 (Python) e itol (https://itol.embl.de/).  

Cuenta microreact (https://microreact.org/), donde puede iniciar sesión con su cuenta de Google.

In [ ]:
!python --version

Python 3.7.15


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!conda install -c conda-forge biopython

In [ ]:
!conda install -c bioconda mafft snp-sites fasttree seqtk snp-dists


### Descargue los datasets para ser analizados
 
Hay tres conjuntos de datos asociados con este módulo. Los primeros dos conjuntos de datos son ejemplos. El último dataset son los datos de ensamblaje. Analizar los conjuntos de datos de ejemplo junto con los videos de instrucciones le dará la experiencia necesaria para analizar el conjunto de datos de evaluación.

In [ ]:
!wget https://wcs_data_transfer.cog.sanger.ac.uk/Module4_data_zip.zip
!unzip Module4_data_zip.zip

### Dataset 1

Primero evaluaremos la calidad del genoma para ver si los genomas son lo suficientemente buenos para el análisis. Para SARS-CoV-2, estamos principalmente interesados en el número de Ns.El paquete que utilizaremos es seqtk comp, que ofrece muchas estadísticas para un archivo de secuencia dado en formato FASTA.

Output format: chr, length, #A, #C, #G, #T, #2, #3, #4, #CpG, #tv, #ts, #CpG-ts

9th column, #4 is the number of Ns, 4 ambiguities, ie A, G, T, or C.

Si ha separado los archivos FASTA, podría ser más fácil combinar los archivos. De lo contrario, debe analizar por los archivos del genoma por separado.

In [ ]:
#cat (concatenate) comando que combina los fastas en un archivo
#seqtk comp Análisis de la información de secuencia
#cut -f 1,9 Selecciona la columna 1 y 9, la información que necesitamos.
!cat Module4_data/dataset_1_flight/20CV*.fasta > dataset_1_flight_combined.fasta
!seqtk comp dataset_1_flight_combined.fasta | cut -f 1,9 > dataset_1_quality.tsv

In [ ]:
!cat dataset_1_quality.tsv

La calidad se ve bien,si emgargo, 20CV0410  no es tan buena como el resto. Tiene 1140 bases ambiguas, pero es lo suficientemente bueno para el análisis. En general, requerimos el 90% o menos de **3000** Ns en el genoma para ensamblar y realizar el análisis de árboles filogenéticos. Sin embargo, hay grupos que tienen requisitos menos estrictos, algunos son tan bajos como 50%. Tener el 70% del genoma ensamblado es el valor predeterminado en Pangolin para la asignación de linaje.

### Alineamiento

Usaremos mafft para el alineamiento. Es bastante rápido y bastante preciso.

Hay muchas opciones para alinear secuencias en mafft.`--auto` es una buena opción en la que el programa en sí elige el algoritmo más eficiente (buen equilibrio entre velocidad y precisión). El alineamiento podría tardar días en alinear las secuencias largas si usa el algoritmo más adecuado. Si está alineando secuencias cortas, como una región del gen/proteína Spike, puede usar opciones más precisas. El sitio web de mafft tiene buenos ejemplos de qué usar y cuando.

Para que mafft alinee las secuencias, en nuestro caso, debe combinar las secuencas que desea alinear en un solo archivo. Lo bueno es que ya lo ha hecho y, dado que todos los genomas son buenos, no necesitamos excluir ninguno de la alineación.

In [ ]:
!mafft --auto dataset_1_flight_combined.fasta > dataset_1_flight_aligned.fasta

### Mirada rápida a las diferencias de los genomas
snp-site es un software realmente bueno para darse una idea de cuán relacionados están sus genomas de interés. Para pequeños conjuntos de datos como este, esto es realmente bueno. Analiza su alineamiento y le da un alineamiento de SNP como predeterminado. También puede brindarle información de los SNPs en formato VCF.

In [ ]:
!snp-sites dataset_1_flight_aligned.fasta

Después de esta mirada rápida, verá que la mayoría de los genomas son indistinguibles, aparte de 20CV0401 que tiene un SNP adicional.

### Interpretación

¿Qué se puede decir a partir de un montón de genomas idénticos?

Como sabemos que estas personas llegaron en el mismo vuelo, podemos analizar uno de los genomas que usan https://genome.ucsc.edu/cgi-bin/hgphyloplace o si tiene una cuenta, puede usar el programa GISAID para encontrar el genoma internacional más cercano. Descargue un genoma e intente esto con su navegador web.

Consulte la publicación https://www.ncbi.nlm.nih.gov/pmc/articles/pmc7920679/ para el análisis de este conjunto de datos.

In [ ]:
!cat Module4_data/dataset_1_flight/dataset_1_flight_metadata.tsv

### Dataset 2

In [ ]:
!seqtk comp Module4_data/dataset_2_hotel/dataset_2_quarantineHotel.fasta | cut -f 1,9 > dataset_2_quality.tsv
!cat dataset_2_quality.tsv
#¿Son los conjuntos del genoma de buena calidad?

In [ ]:
#Los archivos FASTA ya están concatenados.
!mafft --auto Module4_data/dataset_2_hotel/dataset_2_quarantineHotel.fasta > dataset_2_quarantineHotel_aligned.fasta

In [ ]:
!snp-sites dataset_2_quarantineHotel_aligned.fasta

In [ ]:
#puede mirar la distancia de SNP entre pares de muestras usando snp-dist
!snp-dists dataset_2_quarantineHotel_aligned.fasta


La alineación es un poco más complicada con más diferencias, por lo tanto, necesitamos un árbol filogenético para visualizar las relaciones

In [ ]:
!fasttree -nt -gtr -gamma dataset_2_quarantineHotel_aligned.fasta > dataset_2_quarantineHotel_aligned.nwk

In [ ]:
!cat dataset_2_quarantineHotel_aligned.nwk
#Como puede ver, es bastante difícil interpretar este formato sin visualizar en forma de árbol

In [ ]:
from Bio import Phylo

tree = Phylo.read("dataset_2_quarantineHotel_aligned.nwk", "newick")
print("tree")
Phylo.draw_ascii(tree)


Interpretación

Este conjunto de datos se trata de una incursión y se extiende hacia la comunidad desde Quarentine Hotel. ¿Qué podemos decir sobre ese árbol? Siga el video de publicación e instrucciones. Los metadatos asociados con el caso en los encuentra en dataset_2_hotel/dataset_2_quarantineHotel_metadata.tsv.


Consulte la publicación https://www.ncbi.nlm.nih.gov/pmc/articles/pmc8084504/ para el análisis de este conjunto de datos.

Compare el árbol anterior con la publicación, ¿Se ve diferente?

In [ ]:
!cat Module4_data/dataset_2_hotel/dataset_2_quarantineHotel_metadata.tsv

In [ ]:
#let's add the 2020 reference genome to the dataset. 
!mafft --auto --add Module4_data/dataset_2_hotel/dataset_2_quarantineHotel.fasta Module4_data/dataset_2_hotel/references.fasta > dataset_2_quarantineHotel_aligned_ref.fasta

In [ ]:
!fasttree -nt -gtr -gamma dataset_2_quarantineHotel_aligned_ref.fasta > dataset_2_quarantineHotel_aligned_ref.nwk

Ahora, si enraizamos la filogenia usando la referencia, el árbol se parece más a la publicación. ¿Por qué?

Eche un vistazo a la alineación de SNP también usando snp-site. Verá que el uso de la referencia permitió la identificación de bases ancestrales, por lo que ahora sabemos qué cambios en los nucleótidos son mutaciones del tipo salvaje. Anteriormente, el algoritmo de árbol suponía que la base mayoritaria es la secuencia ancestral, que no es correcta.

In [ ]:
from Bio import Phylo

treeOutgroup = Phylo.read("dataset_2_quarantineHotel_aligned_ref.nwk", "newick")
treeOutgroup.root_with_outgroup({"name": "nCoV2019|Wuhan-Hu-1|MN908947|China|Wuhan|2019-12"}) 
print("treeOutgroup")
Phylo.draw_ascii(treeOutgroup)

### Observar la demostración/video de microreact y nextstrain

Mirar los árboles de esta manera es bastante difícil, existen algunas buenas herramientas que permiten árboles interactivos y visualización de metadatos.

https://docs.microreact.org/   
https://nextstrain.org/community/narratives/ESR-NZ/GenomicsNarrativeSARSCoV2/aotearoa-border-incursions

Proyecto de mmicroreact de demostración del dataset 2
https://microreact.org/project/5ELv2rXSKKeZ8XZCFXq9Ug-dataset2hotel


### Otras herramientas para conjuntos de datos y construcción de árboles más grandes
mafft incluso en el modelo de precisión mínima puede ser bastante lento una vez que tenga 1000 de genomas para alinearse. Para acelerar las cosas, puede agregar nuevos genomas a las alineaciones existentes o dividir los genomas en grupos como linajes o sublinajes y luego fusionar las alineaciones. El sitio web de mafft tiene una buena explicación de estos trucos y sus problemas.

nextalign es otra herramienta que puede usar.¡Es súper rápida!(https://docs.nextstrain.org/projects/nextclade/en/stable/user/nextalign-cli.html)

iqtree2 es un paquete integral (no baysiano) de construcción de filogenia que puede probar los mejores modelos de substitución  para su conjunto de datos, realizar análisis bootstrap, comparar filogenia, etc.  Es probablemente una de las mejores herramientas de construcción de árbol.(http://www.iqtree.org/)

usher es un paquete de software que permite agregar secuencias a los árboles existentes sin volver a hacer toda la alineación.(https://www.nature.com/articles/s41588-021-00862-7) Usamos esto cuando analizamos el conjunto de datos de vuelo. Hay algunas advertencias y problemas que debe tener en cuenta. Si la alineación existente está realmente limpia y las secuencias a agregar son de alta calidad, usar usher es una buena forma rápida de obtener algunos resultados preliminares rápidos. Sin embargo, no trabaja muy bien con Ns o gaps.

Para la filogenia bayesiana temporal , a menudo se usa BEAST2. (https://www.beast2.org/) https://www.nature.com/articles/s41559-017-0280-x

### Ahora estamos listos para la tarea


Para esta tarea, investigará un brote potencial en un centro de atención médica.
Los genomas incluidos son de casos que se encuentran en el momento del brote y todos tienen algunos vínculos con el centro de atención médica.Algunos casos han tenido contactos claros con otros casos conocidos (Epi-links to Outbreak:yes) y para algunos contactos no se han encontrado (Epi-links to Outbreak:not found).Y algunos casos son desconocidos (Epi-links to Outbreak: unknown). Su tarea es utilizar el árbol y la fecha de informe para decidir si esos casos etiquetados como desconocidos deben investigarse más a fondo, si están vinculados genómicamente al brote o pueden excluirse del brote.

Necesitas:

1) Revisar la calidad de los genomas proporcionados y excluir cualquier genoma de baja calidad del análisis.
2) Alinear los genomas de buena calidad (**<3000 Ns**) y construir un árbol filogenético.
3) Utilizar el árbol que generaste y los metadatos proporcionados para configurar un proyecto en Microreact para visualizar y examinar el árbol.
4) Usar los metadatos proporcionados para decidir si los aislamientos bajo cuestionamiento están dentro del brote.
5) Responder las preguntas en la evaluación.

**Pistas**:

Puede usar "seqtk subseq" para eliminar secuencias de baja calidad, o puedes mover (mv) las secuencias de baja calidad a otro directorio.

Utilice la distancia de SNP para ver si algunos de los casos son "demasiado" diferentes del resto.

In [ ]:
#Control de calidad de las ensamblajes de genomas para verificar si alguno de ellos no es adecuado para un análisis adicional


In [ ]:
#Lista los aislamientos que crees que pueden no ser adecuados para un análisis adicional a continuación
#Cambia XXX por el número de aislamiento
!echo "these isolates does not meet minimum quality standards: XXX and XXX"

In [ ]:
#Alinea los genomas de buena calidad con mafft


In [ ]:
#Construye un árbol filogenético con el alineamiento de mafft.


In [ ]:
#Visualiza el árbol con Phylo. 
#Recuerda importar Phylo con "from Bio import Phylo" si aún no lo has hecho.


Ahora tiene todos los archivos necesarios para crear un proyecto en Microreact. 
Descargue el árbol y el archivo de metadatos: outbreak_microreact.tsv

**Ahora que tiene el proyecto en Microreact, será más fácil para responder las preguntas de evaluación.**

Opcionalmente, puedes usar `snp-dist` y `snp-site` para ayudarte con el análisis.